In [1]:
!pip install pypsa
!pip install tensorflow==2.2.0
!pip update tensorflow
!pip install tensorflow-data-validation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.6 MB/s eta 0:00:00
  Created wheel for pypsa: filename=pypsa-0.21.3-py2.py3-none-any.whl size=154711 sha256=9bb1efe04a0367a80dd402e18c63b3c54182bfc64d6d289c5de660e42ceeaab4
  Stored in directory: /root/.cache/pip/wheels/32/f0/2e/586eea09cf93b0339455ed1c0e95fbaad62f38a0050e777070
Successfully built pypsa
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 MB 2.1 MB/s eta

In [1]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date, datetime
import tensorflow as tf
import tensorflow_data_validation as tfdv

from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Einlesen der Datenreihen

Jupyter Notebooks

In [3]:
# Erzeugung durch Erneuerbare
# data_pv = pd.read_csv('data/data_pv_antofagasta_1MW.csv', skiprows = 3)['electricity']/1000
# data_wind = pd.read_csv('data/data_wind_antofagasta_1MW.csv', skiprows = 3)['electricity']/1000
# data_electrolysis = pd.read_excel('data/load_electrolysis.xlsx')['hydrogen']
# data_shipping_1 = pd.read_excel('data/Ferntransport.xlsx')['Kapazität [MWh]']
# data_shipping_2 = 0

Google Colab mit Drive Access

In [4]:
#infeed renewables
data_pv = pd.read_csv('/content/drive/MyDrive/DWE_Projektarbeit_2022/data_pv_antofagasta_1MW.csv', skiprows = 3)
data_wind = pd.read_csv('/content/drive/MyDrive/DWE_Projektarbeit_2022/data_wind_antofagasta_1MW.csv', skiprows = 3)
data_electrolysis = pd.read_excel('/content/drive/MyDrive/DWE_Projektarbeit_2022/load_electrolysis.xlsx')
data_shipping_1 = pd.read_excel('/content/drive/MyDrive/DWE_Projektarbeit_2022/Ferntransport.xlsx')
data_shipping_2 = 0

FileNotFoundError: ignored

In [6]:
#infeed renewables
data_pv = pd.read_csv('/content/drive/MyDrive/DWE_Projektarbeit_2022/data_pv_antofagasta_1MW.csv', skiprows = 3)
data_wind = pd.read_csv('/content/drive/MyDrive/DWE_Projektarbeit_2022/data_wind_antofagasta_1MW.csv', skiprows = 3)
data_electrolysis = pd.read_excel('/content/drive/MyDrive/DWE_Projektarbeit_2022/load_electrolysis.xlsx')
data_shipping_1 = pd.read_excel('/content/drive/MyDrive/DWE_Projektarbeit_2022/Ferntransport.xlsx')
data_shipping_2 = 0

In [ ]:
dataset_df=pd.DataFrame()
dataset_df['Zeitstempel']=data_pv['time']
dataset_df['PV_Erzeugung']=data_pv['electricity']
dataset_df['Wind_Erzeugung']=data_wind['electricity']
dataset_df['Elektrolyse']=data_electrolysis['hydrogen']
dataset_df['Kapazität_Schiff']=data_shipping_1['Kapazität [MWh]']
dataset_df.Kapazität_Schiff=dataset_df.Kapazität_Schiff.fillna(method='ffill')
dataset_df.head(10000)

In [ ]:
#Vorverarbeitung für Tensorflow Data Validation 
features_to_remove = {}
approved_cols = [col for col in dataset_df.columns if (col not in features_to_remove)]
stats_options = tfdv.StatsOptions(feature_allowlist=approved_cols)

dataset_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset_df,stats_options = stats_options)
print(f"Anzahl der verwendeten Parameter: {len(dataset_stats.datasets[0].features)}")
print(f"Anzahl der verwendeten Trainingsbeispiele: {dataset_stats.datasets[0].num_examples}")
tfdv.visualize_statistics(dataset_stats)

In [ ]:
# Inputs Variablen
data_inputs = pd.read_excel('/content/drive/MyDrive/DWE_Projektarbeit_2022/Inputs_Modell_1.xlsx', sheet_name = 'Gesamt', header = 2, index_col = 0)

data_inputs=pd.DataFrame(data_inputs)
data_inputs.head()

In [ ]:
# Erstellung Last
hydrogen_energy_content = 33 #kWh/kg
hydrogen_demand = (35 * 1000) * hydrogen_energy_content # MWh
hydrogen_demand_hourly = hydrogen_demand/8760 #MW

print('Der stündliche Bedarf bei konstanter Verteilung des Jahresbedarfs von 35 kt Wasserstoff beträgt ungefähr ' + str(round(hydrogen_demand_hourly,0)) + ' MWh')

#Erstellung Elektrolyseprofil

# Definition der Variablen

## Leistungen

In [ ]:
p_nom_pv = data_pv.max()
p_nom_wind = data_wind.max()

## Investitionskosten

In [ ]:
# generators
capital_cost_pv = 800 # €/MW
capital_cost_wind = 800 # €/MW

# links
capital_cost_PEMEL = data_inputs.loc['PEM Elektrolyse']['capital_cost'] # €/MW
capital_cost_AEL = data_inputs.loc['AEL Elektrolyse']['capital_cost'] # €/MW
#capital_cost_HTEL = 1300 # €/MW
capital_cost_NH3 = 1400 # €/MW
capital_cost_LH2 = data_inputs.loc['Verflüssigung']['capital_cost'] # €/MW
capital_cost_LHOC = 1600 # €/MW
capital_cost_pipeline = 5000 # €/MW
capital_cost_shipping_1 = 10000 # €/MW
capital_cost_shipping_2 = 80000 # €/MW

# storages
capital_cost_storage_GH2 = 200 # €/MW
capital_cost_storage_NH3 = 20 # €/MW
capital_cost_storage_LH2 = data_inputs.loc['LH2 Speicher']['capital_cost'] # €/MW
capital_cost_storage_LHOC = 20 # €/MW

## Gestehungskosten

In [ ]:
# generators
marginal_cost_pv = 70 # €/MWh
marginal_cost_wind = 70 # €/MWh

# conversion
marginal_cost_PEMEL = data_inputs.loc['PEM Elektrolyse']['marginal_cost'] # €/MWh
marginal_cost_AEL = data_inputs.loc['AEL Elektrolyse']['marginal_cost'] # €/MWh
#marginal_cost_HTEL = 100 # €/MWh
marginal_cost_NH3 = 100 # €/MWh
marginal_cost_LH2 = data_inputs.loc['Verflüssigung']['marginal_cost'] # €/MWh
marginal_cost_LHOC = 100 # €/MWh

# storage
marginal_cost_storage_GH2 = 20 # €/MWh
marginal_cost_storage_NH3 = 20 # €/MWh
marginal_cost_storage_LH2 = 20 # €/MWh
marginal_cost_storage_LHOC = 20 # €/MWh

# transportation
marginal_cost_pipeline = 200 # €/MWh
marginal_cost_shipping_GH2 = 10 # €/MWh
marginal_cost_shipping_NH3 = 10 # €/MWh
marginal_cost_shipping_LH2 = data_inputs.loc['LH2 Speicher']['marginal_cost'] # €/MWh
marginal_cost_shipping_LHOC = 10 # €/MWh

## Sonstiges

In [ ]:
# efficiencies
efficiency_PEMEL = data_inputs.loc['PEM Elektrolyse']['efficiency']
efficiency_AEL = data_inputs.loc['AEL Elektrolyse']['efficiency']
#efficiency_HTEL = 1
efficiency_NH3 = 1
efficiency_LH2 = data_inputs.loc['Verflüssigung']['efficiency']
efficiency_LHOC = 1

# lifetimes
lifetime_PEMEL = 20
lifetime_AEL = 20
#lifetime_HTEL = 20

# standing losses
standing_loss_GH2 = 0.05
standing_loss_NH3 = 0.05
standing_loss_LH2 = data_inputs.loc['LH2 Speicher']['standing_loss']
standing_loss_LHOC = 0.05

# Netzwerk aufbauen

In [ ]:
network = pypsa.Network()
network.set_snapshots(range(8760))

## Buses

In [ ]:
network.add('Bus', name = 'electricity')
network.add('Bus', name = 'GH2')
network.add('Bus', name = 'LH2')
#network.add('Bus', name = 'NH3')
#network.add('Bus', name = 'LH2')
#network.add('Bus', name = 'LHOC')
#network.add('Bus', name = 'hydrogen')

network.add('Bus', name = 'bremerhaven')
#network.add('Bus', name = 'cologne')

## Loads

In [ ]:
network.add('Load', name = 'hydrogen_demand', bus = 'bremerhaven', p_set = hydrogen_demand_hourly)

## Generators

In [ ]:
network.add('Generator', name = 'pv', bus = 'electricity', 
            p_nom = p_nom_pv, p_max_pu = data_pv/p_nom_pv, p_min_pu = data_pv/p_nom_pv, p_nom_extendable = True,
            marginal_cost = marginal_cost_pv, capital_cost = capital_cost_pv)

#network.add('Generator', name = 'pv', bus = 'electricity', 
#            p_nom = p_nom_pv, p_max_pu = data_pv/p_nom_pv, p_nom_extendable = True,
#            marginal_cost = marginal_cost_pv, capital_cost = capital_cost_pv)

network.add('Generator', name = 'wind', bus = 'electricity',
            p_nom = p_nom_wind, p_max_pu = data_wind/p_nom_wind, p_min_pu = data_wind/p_nom_wind, p_nom_extendable = True,
            marginal_cost = marginal_cost_wind, capital_cost = capital_cost_wind)

## Links - Electrolysis

In [ ]:
network.add('Link', name = 'PEMEL', bus0 = 'electricity', bus1 = 'GH2', 
            p_nom_extendable = True,
            marginal_cost = marginal_cost_PEMEL, capital_cost = capital_cost_PEMEL,
            efficiency = efficiency_PEMEL)


network.add('Link', name = 'AEL', bus0 = 'electricity', bus1 = 'GH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_AEL, capital_cost = capital_cost_AEL,
           efficiency = efficiency_AEL)
network.add('Link', name = 'HTEL', bus0 = 'electricity', bus1 = 'GH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_HTEL, capital_cost = capital_cost_HTEL,
           efficiency = efficiency_HTEL)

## Links - Conversion

In [ ]:
network.add('Link', name = 'LH2', bus0 = 'GH2', bus1 = 'LH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_LH2, capital_cost = capital_cost_LH2,
           efficiency = efficiency_LH2)

network.add('Link', name = 'NH3', bus0 = 'electricity', bus1 = 'NH3',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_NH3, capital_cost = capital_cost_NH3,
           efficiency = efficiency_NH3)

network.add('Link', name = 'LHOC', bus0 = 'electricity', bus1 = 'LHOC',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_LHOC, capital_cost = capital_cost_LHOC,
           efficiency = efficiency_LHOC)

## Links - Ferntransport

In [ ]:
# long distance shipment
network.add('Link', name = 'Ferntransport_GH2', bus0 = 'LH2', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(), p_nom_extendable = True,
            marginal_cost = marginal_cost_shipping_GH2, capital_cost = capital_cost_shipping_1)

network.add('Link', name = 'Ferntransport_NH3', bus0 = 'NH3', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_NH3)

network.add('Link', name = 'Ferntransport_LH2', bus0 = 'LH2', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_LH2)

network.add('Link', name = 'Ferntransport_LHOC', bus0 = 'LHOC', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_LHOC)


## Links - Pipeline

## Storages

In [ ]:
network.add('Store', name = 'antofagasta_GH2', bus = 'GH2', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_GH2)

network.add('Store', name = 'hydrogen_store_1.1', bus = 'NH3', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_NH3)

network.add('Store', name = 'hydrogen_store_1.2', bus = 'LH2', 
            e_nom_extendable = True, e_cyclic = True,  
            capital_cost = capital_cost_storage_LH2)

network.add('Store', name = 'hydrogen_store_1.3', bus = 'LHOC', 
            e_nom_extendable = True, e_cyclic = True,  
            capital_cost = capital_cost_storage_LHOC)

network.add('Store', name = 'bremerhaven_GH2', bus = 'bremerhaven', 
            e_nom_extendable = True, e_cyclic = True, 
            capital_cost = capital_cost_storage_GH2)

network.add('Store', name = 'cologne_GH2', bus = 'cologne', 
            e_nom_extendable = True, e_cyclic = True, 
            capital_cost = capital_cost_storage_GH2)

# Optimierung des Systems

In [ ]:
network.lopf(pyomo = False, solver_name = 'gurobi')

# Auswertung der Ergebnisse

In [ ]:
def Jahresverlauf (Titel, Zeitstempel, Leistung, xlabel, ylabel):

  fig = plt.figure(figsize=(19,8))

 

  ax1 = fig.add_axes([1, 1, 1, 1])
  ax1.set_title(Titel,weight='bold',fontsize=15)
  ax1.plot(Zeitstempel,Leistung,c='green')
  plt.xlabel(xlabel,fontsize=15 )
  plt.ylabel(ylabel,fontsize=15)

 

  plt.show()

In [ ]:
Jahresverlauf('Stromerzeugung aus Windkraft',dataset_df.index, dataset_df['PV_Erzeugung'],'Zeit','Leistung')

In [ ]:
Jahresverlauf('Stromerzeugung aus Windkraft',dataset_df.index, dataset_df['Wind_Erzeugung'],'Zeit','Leistung')

In [ ]:
Jahresverlauf('Stromerzeugung aus Windkraft',dataset_df.index, dataset_df['Elektrolyse'],'Zeit','Leistung')

In [ ]:
Jahresverlauf('Stromerzeugung aus Windkraft',dataset_df.index, dataset_df['Kapazität_Schiff'],'Zeit','Leistung')

In [ ]:
network.generators.p_nom_opt

In [ ]:
(network.generators_t.p).plot(figsize = (15,8), ylabel = 'Leistung in MW')

In [ ]:
network.links.p_nom_opt

In [ ]:
(network.links_t.p0).plot(figsize = (15,8), ylabel = 'Leistung in MW')

In [ ]:
network.stores_t.e.plot(figsize = (15,8))

# Werkstatt

In [ ]:
Speicherkosten = network.stores.e_nom_opt*network.stores.capital_cost
spez_Speicherkosten_MWh = Speicherkosten/hydrogen_demand
spez_Speicherkosten_kg = spez_Speicherkosten_MWh/hydrogen_energy_content
spez_Speicherkosten_kg

In [ ]:
labels = ['A', 'B']
Stromkosten = np.array([1.5,1.5])
Elektrolyse = np.array([0.5,1.5])
Speicherung = np.array([0.15,0.12])
Ferntransport_1 = np.array([1,0.2])
Inlandstransport = np.array([0.3,0.3])
Umwandlung = np.array([0.7,0.3])
Sonstige= np.array([0.1,0.3])
# the width of the bars: can also be len(x) sequence
width=0.5
fig, ax = plt.subplots(figsize=(7,10))
#x_pos = [-2,0,2]
ax.bar(labels, Stromkosten, label='Stromkosten',width=width,color='lightcoral')
ax.bar(labels, Elektrolyse, bottom=Stromkosten,
       label='Elektrolyse',width=width,color='indianred')
ax.bar(labels, Speicherung, bottom=Stromkosten+Elektrolyse,
       label='Speicherung',width=width,color='firebrick')
ax.bar(labels, Ferntransport_1, bottom=Stromkosten+Elektrolyse+Speicherung,
       label='Ferntransport',width=width,color='brown')
ax.bar(labels, Inlandstransport, bottom=Stromkosten+Elektrolyse+Speicherung+Ferntransport_1,
       label='Inlandstransport',width=width,color='maroon')
ax.bar(labels, Umwandlung, bottom=Stromkosten+Elektrolyse+Speicherung+Ferntransport_1+Inlandstransport,
       label='Umwandlung',width=width,color='darkred')
ax.bar(labels, Sonstige, bottom=Stromkosten+Elektrolyse+Speicherung+Ferntransport_1+Inlandstransport+Umwandlung,
       label='Sonstige',width=width,color='red')

ax.set_ylabel('Kosten in [€]')
ax.set_title('Wasserstoffbereitstellungskosten in €/kg Wasserstoff')
ax.legend(loc='upper right')

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, 
            y+height/2, 
            '{:.02f}€'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',
            color='white',
            fontsize=14,
            )
  

horiz_offset = 1.03
vert_offset = 1.
ax.legend(bbox_to_anchor=(horiz_offset, vert_offset))
#plt.xticks(x_pos, labels)
plt.show()